In [0]:
# 04_load_gold_to_azure_sql
# - UC GOLD -> Azure SQL (2 tablas)
# - Usa Key Vault secrets (scope + keys)
# - Modo: overwrite / append

#WIDGETS

In [0]:
dbutils.widgets.removeAll()

dbutils.widgets.text("uc_catalog", "catalog_project")
dbutils.widgets.text("gold_schema", "gold")

dbutils.widgets.text("sql_server", "sqlsrv-etlproject.database.windows.net")
dbutils.widgets.text("sql_database", "sqldb_etlproject")
dbutils.widgets.text("sql_schema", "dbo")

dbutils.widgets.text("secret_scope", "kv_project_etl_03")
dbutils.widgets.text("secret_user_key", "sqluser")
dbutils.widgets.text("secret_password_key", "sqlpassword1")

dbutils.widgets.dropdown("load_mode", "overwrite", ["overwrite", "append"])



In [0]:
uc_catalog  = dbutils.widgets.get("uc_catalog").strip()
gold_schema = dbutils.widgets.get("gold_schema").strip()

sql_server   = dbutils.widgets.get("sql_server").strip()
sql_database = dbutils.widgets.get("sql_database").strip()
sql_schema   = dbutils.widgets.get("sql_schema").strip()

secret_scope = dbutils.widgets.get("secret_scope").strip()
user_key     = dbutils.widgets.get("secret_user_key").strip()
pwd_key      = dbutils.widgets.get("secret_password_key").strip()

load_mode    = dbutils.widgets.get("load_mode").strip()


#SECRETS (KEY VAULT)

In [0]:
sql_user = dbutils.secrets.get(secret_scope, user_key)
sql_pwd  = dbutils.secrets.get(secret_scope, pwd_key)

#CONEXION AZURE SQL

In [0]:
jdbc_url = (
    f"jdbc:sqlserver://{sql_server}:1433;"
    f"database={sql_database};"
    "encrypt=true;"
    "trustServerCertificate=false;"
    "hostNameInCertificate=*.database.windows.net;"
    "loginTimeout=30;"
)
driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"


In [0]:
spark.read.format("jdbc") \
  .option("url", jdbc_url) \
  .option("query", "SELECT 1 AS ok") \
  .option("user", sql_user) \
  .option("password", sql_pwd) \
  .option("driver", driver) \
  .load() \
  .show()

In [0]:
def write_to_sql(df, table_name: str):
    target = f"{sql_schema}.{table_name}"
    (df.write
        .mode(load_mode)
        .format("jdbc")
        .option("url", jdbc_url)
        .option("dbtable", target)
        .option("user", sql_user)
        .option("password", sql_pwd)
        .option("driver", driver)
        .save()
    )
    print(f"✅ Loaded: {target} (mode={load_mode})")

#LOAD GOLD TABLES FROM UC

In [0]:
gold_hybrid_table  = f"{uc_catalog}.{gold_schema}.gold_hybrid_manufacturing"
gold_defects_table = f"{uc_catalog}.{gold_schema}.gold_manufacturing_defects"

df_gold_hybrid  = spark.table(gold_hybrid_table)
df_gold_defects = spark.table(gold_defects_table)

print(f"UC Source: {gold_hybrid_table}  -> Azure SQL: {sql_schema}.gold_hybrid_manufacturing")
print(f"UC Source: {gold_defects_table} -> Azure SQL: {sql_schema}.gold_manufacturing_defects")


#WRITE TO AZURE SQL

In [0]:
write_to_sql(df_gold_hybrid,  "gold_hybrid_manufacturing")
write_to_sql(df_gold_defects, "gold_manufacturing_defects")

print("✅ Done.")